**Redimensionamiento de imagenes**

Lo que haremos sera a partir del Dataset STL-10 el cual contiene imagenes de 96 x 96 pixeles reducir dichas imagenes a 66 x 96 piexeles para verificar si la presicion o validation accuracy se mantiene similar, aumenta o disminute. Esto con el proposito de verificar si rel redimensionamiento de imagenes afecta en el entrenamiento.

In [1]:

import torch
import numpy as np
import pandas as pd
import torch.nn as nn

from torchvision import models,datasets,transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from torchsummary import summary
from torchvision.utils import make_grid
from torchvision.datasets import STL10
import itertools
import torch.optim as optim
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve

In [2]:
# transform = transforms.Compose([
# 	transforms.ToTensor(),
# ])


In [3]:
# trainData = STL10('./data', split='train', transform=transform, download=True)
# testData = STL10('./data', split='test', transform=transform, download=True)

trainData = STL10('./data', split='train', download=True)
testData = STL10('./data', split='test', download=True)

100%|██████████| 2640397119/2640397119 [00:49<00:00, 53224326.69it/s]


Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [4]:
#aqui
train_images = np.array(trainData.data)  # Asegurarnos que sean ndarrays de numpy
train_labels = trainData.labels
test_labels = testData.labels
test_images = np.array(testData.data)

print(train_images.shape)
print(train_labels.shape)
print(test_labels.shape)
print(test_images.shape)

(5000, 3, 96, 96)
(5000,)
(8000,)
(8000, 3, 96, 96)


In [5]:
print(trainData[110][0])

<PIL.Image.Image image mode=RGB size=96x96 at 0x7E3F81BC1F00>


In [6]:
print("Train Dataset Size: ", len(trainData))
print("Shape: ", trainData.data.shape)
print("Test Dataset Size: ", len(testData))
print("Shape: ", testData.data.shape)

Train Dataset Size:  5000
Shape:  (5000, 3, 96, 96)
Test Dataset Size:  8000
Shape:  (8000, 3, 96, 96)


dos bacth uno de 5000 para entrenamiento y otro de 8000 para prueba,3 canalaes, 96 alto y 96 ancho

Dataset contiene 13000 imagenes clasificadas en 10 catergorias o etiquetas.

In [7]:
classes = trainData.classes
print("Number of classes = ", len(classes))
print(classes)

Number of classes =  10
['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']


In [10]:
classCount = {}
for i,j in trainData:
    labels = classes[j]
    if labels not in classCount:
        classCount[labels] = 0
    classCount[labels] += 1
classCount

{'bird': 500,
 'dog': 500,
 'horse': 500,
 'cat': 500,
 'truck': 500,
 'monkey': 500,
 'deer': 500,
 'ship': 500,
 'airplane': 500,
 'car': 500}

In [11]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y, trans, device):
        self.X = X
        self.y = y
        self.trans = trans
        self.device = device

    def __len__(self):
        return len(self.X)

    def __getitem__(self, ix):
      img = self.X[ix]

      # Si img es un numpy ndarray, conviértelo en tensor de PyTorch
      if isinstance(img, np.ndarray):
          img = torch.from_numpy(img)

      # Convertir a formato [altura, anchura, canales]
      img = img.permute(1, 2, 0)
      #print("Shape después de cargar:", img.shape)

      # Aplicar transformaciones de albumentations
      if self.trans:
          img = self.trans(image=img.numpy())["image"]
      #print("Shape después de transformar:", img.shape)

      # Convertir a tensor (si no lo es ya) y reordenar a formato [canales, altura, anchura]
      if isinstance(img, np.ndarray):
          img = torch.from_numpy(img)
      img = img.float().permute(2, 0, 1)
      #print("Shape después de permute:", img.shape)

      return img, torch.tensor(self.y[ix])

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
import albumentations as A

# Transformaciones de albumentations
trans = A.Compose([
    A.Resize(66,96)
])

dataset = {
    'train': Dataset(train_images, train_labels, trans, device),
    'test': Dataset(test_images, test_labels, trans, device)
}

print("shape original:", trainData.data[0].shape)

shape original: (3, 96, 96)


In [ ]:
# #version git
# batch_size = 100
# trainLoader = DataLoader(trainData, batch_size=batch_size, shuffle=True,num_workers=2,pin_memory =True)
# testLoader = DataLoader(testData, batch_size=batch_size, shuffle=True,num_workers =2,pin_memory =True)

In [16]:
dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=64, shuffle=True, pin_memory=True),
    'test': torch.utils.data.DataLoader(dataset['test'], batch_size=256, shuffle=False)
}

imgs, labels = next(iter(dataloader['train']))
print("shape después de transformaciones:", imgs.shape)

shape después de transformaciones: torch.Size([64, 3, 66, 96])


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

TEST

In [19]:
import torchvision

resnet = torchvision.models.resnet50()
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

real test

In [21]:
import torchvision

class ModelCustom(torch.nn.Module):
  def __init__(self, n_outputs=10, pretrained=True, freeze=False):
    super().__init__()
    # descargamos resnet
    resnet = torchvision.models.resnet50(pretrained=pretrained)
    # nos quedamos con todas las capas menos la última
    self.resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
    if freeze:
      for param in self.resnet.parameters():
        param.requires_grad=False
    # añadimos una nueva capa lineal para llevar a cabo la clasificación
    self.fc = torch.nn.Linear(2048, 10)

  def forward(self, x):
    x = self.resnet(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

  def unfreeze(self):
    for param in self.resnet.parameters():
        param.requires_grad=True

In [22]:
model_custom = ModelCustom()
outputs = model_custom(torch.randn(64, 3, 66, 96))
outputs.shape


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 68.2MB/s]


torch.Size([64, 10])

In [23]:
from tqdm import tqdm
import numpy as np

def fit(model, dataloader, epochs=5, lr=1e-2):
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [25]:
#transfer learning a resnet50
model_c = ModelCustom(pretrained=True, freeze=False)
fit(model_c, dataloader)
#Con pretrained = true y freeze = true:  acc 0.75198 val_acc 0.71619

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
val_loss 0.53867 val_acc 0.81946: 100%|██████████| 32/32 [00:05<00:00,  5.80it/s]


Epoch 1/5 loss 1.11457 val_loss 0.53867 acc 0.66693 val_acc 0.81946


val_loss 0.41489 val_acc 0.85620: 100%|██████████| 32/32 [00:05<00:00,  5.46it/s]


Epoch 2/5 loss 0.35580 val_loss 0.41489 acc 0.89339 val_acc 0.85620


val_loss 0.39493 val_acc 0.86450: 100%|██████████| 32/32 [00:06<00:00,  5.25it/s]


Epoch 3/5 loss 0.15035 val_loss 0.39493 acc 0.95985 val_acc 0.86450


val_loss 0.39204 val_acc 0.86621: 100%|██████████| 32/32 [00:06<00:00,  5.28it/s]


Epoch 4/5 loss 0.07841 val_loss 0.39204 acc 0.98418 val_acc 0.86621


val_loss 0.35251 val_acc 0.88293: 100%|██████████| 32/32 [00:06<00:00,  4.68it/s]

Epoch 5/5 loss 0.04743 val_loss 0.35251 acc 0.99150 val_acc 0.88293


Con transfer learning y con fine tunning la presicion de entrenamiento obtenida es: acc 0.99150 val_acc 0.88293, un muy buen valor. Ahora, comparando estos resultados con otro modelo que utilize el mismo dataset pero que no reduzca el tamaño de las imagenes (aplicando  tambien transfer learning y fine tunning) se obtuvieron los resultados de acc 0.99506 val_acc 0.92456(anexado en el link)
Se peude comprobar que sin reducir la imagen se obtuvo ligeramente mejor resultado en el validation accuracy.

In [27]:
model_o = ModelCustom(pretrained=True, freeze=True)
fit(model_o, dataloader)
model_o.unfreeze()
fit(model_o, dataloader, lr=1e-3)

val_loss 1.17695 val_acc 0.64978: 100%|██████████| 32/32 [00:06<00:00,  5.32it/s]


Epoch 1/5 loss 1.53976 val_loss 1.17695 acc 0.54035 val_acc 0.64978


val_loss 1.01440 val_acc 0.66687: 100%|██████████| 32/32 [00:06<00:00,  5.29it/s]


Epoch 2/5 loss 1.00507 val_loss 1.01440 acc 0.70273 val_acc 0.66687


val_loss 0.91885 val_acc 0.69568: 100%|██████████| 32/32 [00:05<00:00,  5.41it/s]


...

Epoch 3/5 loss 0.45341 val_loss 0.58389 acc 0.86669 val_acc 0.80273


val_loss 0.55365 val_acc 0.80908: 100%|██████████| 32/32 [00:05<00:00,  5.63it/s]


Epoch 4/5 loss 0.39292 val_loss 0.55365 acc 0.89221 val_acc 0.80908


val_loss 0.53677 val_acc 0.81897: 100%|██████████| 32/32 [00:05<00:00,  5.58it/s]

Epoch 5/5 loss 0.34779 val_loss 0.53677 acc 0.90348 val_acc 0.81897


Tambien podemos ver que aplicando estrategias de congelar y descongelar la modificacion de los pesos (que resulta ser eficiente en algunos casos) tambien el dataset que no se  modifico el tamaño de las imagenes obtiene mejor resultado. Se Obtuvo en ese caso un acc 0.94225 val_acc 0.87854 y en este caso acc 0.90348 val_acc 0.81897

Tratando de mejorar la presicion del modelo obtenemos una reduccion en la misma

In [ ]:
# # newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))
# # summary(newmodel, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

(5000, 2048)
Length of Y_train 5000


(8000, 2048)
Length of Y_test 8000


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: 

5000

Accuracy:  0.8286


Confusion matrix

Accuracy:  0.097875
